# Import packages

In [7]:
# /data_ssd/ysQiu/anaconda3/envs/scDOT/bin/python

import pandas as pd
import numpy as np
from scDOT_modified import scDOT

# Load file names of data

In [8]:
batch = pd.read_csv(filepath_or_buffer="/data_ssd/ysQiu/TanWeiqiang/scDOT/datasets/pbmc/pbmc/batch.csv", header=0, index_col=False).x.values
query_name = batch[0]
# ref_name = batch[1:5] # 全部读取内存不足
# ref_name = batch[1:] # 全部读取
# """
ref_name = batch[[
                    # 0, 
                    1, 
                    2, 
                    3, 
                    4, 
                    5, 
                    6, 
                    7, 
                    8, 
                    9, 
                    10, 
                    11
                ]]
# """
query_name

'Chromium'

In [3]:
# batch = pd.read_csv(filepath_or_buffer="/data_ssd/ysQiu/TanWeiqiang/mtANN/pancreas/batch.csv", header=0, index_col=False).x.values
# query_name = batch[3]
# # ref_name = batch[1:5] # 全部读取内存不足
# # ref_name = batch[1:] # 全部读取
# # """
# ref_name = batch[[
#                     0, 
#                     1, 
#                     2, 
#                     # 3, 
#                 ]]
# # """
# query_name

In [4]:
# name: str = batch[4]
# file_name = "./datasets/pbmc/pbmc/" + f"{name}_label.csv"
# y_true = pd.read_csv(filepath_or_buffer=file_name, header=0, index_col=0).iloc[:,0].values
# tmp = pd.DataFrame(data=y_true)
# print(name)
# tmp[0].unique()

In [9]:
ref_name

array(['inDrop', 'C1HT-medium', 'C1HT-small', 'CEL-Seq2', 'ddSEQ',
       'Drop-Seq', 'ICELL8', 'MARS-Seq', 'Quartz-Seq2', 'mcSCRB-Seq',
       'Smart-Seq2'], dtype=object)

# 阈值敏感程度

In [10]:
threshold: float
for threshold in range(1, 10):
    threshold /= 10
    M, x, y_hat, score, y_hat_with_unseen = scDOT(
                                                loc="/data_ssd/ysQiu/TanWeiqiang/scDOT/datasets/pbmc/pbmc/",
                                                # loc = "/data_ssd/ysQiu/TanWeiqiang/mtANN/pancreas/",
                                                ref_name=ref_name, 
                                                query_name=query_name,
                                                threshold=threshold
                                            )
    
    file_name = "/data_ssd/ysQiu/TanWeiqiang/scDOT/datasets/pbmc/pbmc/" + f"{query_name}_label.csv"
    # file_name = "/data_ssd/ysQiu/TanWeiqiang/mtANN/pancreas/" + f"{query_name}_label.csv"
    y_true = pd.read_csv(filepath_or_buffer=file_name, header=0, index_col=0).iloc[:,0].values # 原数据集用
    # y_true = pd.read_csv(filepath_or_buffer=file_name, header=0, index_col=False).iloc[:,0].values # 换数据集用
    accuracy1 = np.sum(y_hat == y_true) / y_true.shape[0]
    accuracy2 = np.sum(y_hat_with_unseen == y_true) / y_true.shape[0]
    print(f"The annotation accuracy of {query_name} is {accuracy1, accuracy2}")

    from sklearn.cluster import KMeans
    # import numpy as np
    import collections

    n: int = len(set(y_hat_with_unseen))

    y_pred: np.ndarray = y_true.copy()
    # 应用 K-means 算法
    kmeans = KMeans(n_clusters=n, random_state=314)
    kmeans.fit(X=x)
    y_kmeans = kmeans.labels_

    # 对于每个簇，统计每个实际类型出现的次数
    for cluster_id in range(n):
        # 获取当前簇的所有样本索引
        cluster_samples = np.where(y_kmeans == cluster_id)[0]
        # 获取这些样本的实际类型
        cluster_labels = y_hat_with_unseen[cluster_samples]
        
        # 统计每个类型出现的次数
        label_counts = collections.Counter(cluster_labels)
        
        # 找到出现次数最多的类型
        most_common_label = label_counts.most_common(1)[0][0]
        
        # 将这个类型赋值给簇中的所有样本
        y_pred[cluster_samples] = most_common_label

    # 此时 y_hat_with_unseen 包含了每个样本根据簇中出现次数最多的实际类型重新分类的结果
    res = pd.DataFrame(data={"pred2": y_pred, "pred1": y_hat_with_unseen, "true": y_true})
    unseen_res = res[res["pred1"] == "unseen"]
    # unseen_res = res

    accuracy2 = (np.sum(y_hat_with_unseen == y_true) + np.sum(unseen_res['pred2'] == unseen_res['true'])) / y_true.shape[0]
    # accuracy2 = np.sum(unseen_res['pred2'] == unseen_res['true']) / y_true.shape[0]
    print(f"阈值为：{threshold}, The annotation accuracy of {query_name} is {accuracy1, accuracy2}")

Time for data loading is 107.68253445625305
precessing 0-th data


/data_ssd/ysQiu/anaconda3/envs/scDOT/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:84: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/data_ssd/ysQiu/anaconda3/envs/scDOT/lib/python3.12/site-packages/scanpy/preprocessing/_scale.py:318: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


precessing 1-th data


/data_ssd/ysQiu/anaconda3/envs/scDOT/lib/python3.12/site-packages/scanpy/preprocessing/_scale.py:318: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


precessing 2-th data


...

precessing 10-th data


/data_ssd/ysQiu/anaconda3/envs/scDOT/lib/python3.12/site-packages/scanpy/preprocessing/_scale.py:318: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


precessing 11-th data


/data_ssd/ysQiu/anaconda3/envs/scDOT/lib/python3.12/site-packages/scanpy/preprocessing/_scale.py:318: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


-------------------------------
单个标签的运输成本矩阵形状 (1, 1604)
单个标签的运输成本矩阵形状 (1, 1604)
单个标签的运输成本矩阵形状 (1, 1604)
单个标签的运输成本矩阵形状 (1, 1604)
单个标签的运输成本矩阵形状 (1, 1604)
单个标签的运输成本矩阵形状 (1, 1604)
单个标签的运输成本矩阵形状 (1, 1604)
单个标签的运输成本矩阵形状 (1, 1604)
单个标签的运输成本矩阵形状 (1, 1604)
-------------------------------
单个标签的运输成本矩阵形状 (1, 1604)
单个标签的运输成本矩阵形状 (1, 1604)
单个标签的运输成本矩阵形状 (1, 1604)
单个标签的运输成本矩阵形状 (1, 1604)
单个标签的运输成本矩阵形状 (1, 1604)
单个标签的运输成本矩阵形状 (1, 1604)
单个标签的运输成本矩阵形状 (1, 1604)
单个标签的运输成本矩阵形状 (1, 1604)
单个标签的运输成本矩阵形状 (1, 1604)
-------------------------------
单个标签的运输成本矩阵形状 (1, 1604)
单个标签的运输成本矩阵形状 (1, 1604)
单个标签的运输成本矩阵形状 (1, 1604)
单个标签的运输成本矩阵形状 (1, 1604)
单个标签的运输成本矩阵形状 (1, 1604)
单个标签的运输成本矩阵形状 (1, 1604)
单个标签的运输成本矩阵形状 (1, 1604)
单个标签的运输成本矩阵形状 (1, 1604)
单个标签的运输成本矩阵形状 (1, 1604)
-------------------------------
单个标签的运输成本矩阵形状 (1, 1604)
单个标签的运输成本矩阵形状 (1, 1604)
单个标签的运输成本矩阵形状 (1, 1604)
单个标签的运输成本矩阵形状 (1, 1604)
单个标签的运输成本矩阵形状 (1, 1604)
单个标签的运输成本矩阵形状 (1, 1604)
单个标签的运输成本矩阵形状 (1, 1604)
单个标签的运输成本矩阵形状 (1, 1604)
单个标签的运输成本矩阵形状 (1, 1604)
--------

# Run scDOT

In [ ]:
M, x, y_hat, score, y_hat_with_unseen = scDOT(
                                                # loc="/data_ssd/ysQiu/TanWeiqiang/scDOT/datasets/pbmc/pbmc/",
                                                loc = "/data_ssd/ysQiu/TanWeiqiang/mtANN/pancreas/",
                                                ref_name=ref_name, 
                                                query_name=query_name
                                            )

Time for data loading is 31.004282474517822
precessing 0-th data


/data_ssd/ysQiu/anaconda3/envs/scDOT/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:84: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/data_ssd/ysQiu/anaconda3/envs/scDOT/lib/python3.12/site-packages/scanpy/preprocessing/_scale.py:318: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


precessing 1-th data


/data_ssd/ysQiu/anaconda3/envs/scDOT/lib/python3.12/site-packages/scanpy/preprocessing/_scale.py:318: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


precessing 2-th data


/data_ssd/ysQiu/anaconda3/envs/scDOT/lib/python3.12/site-packages/scanpy/preprocessing/_scale.py:318: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


precessing 3-th data


/data_ssd/ysQiu/anaconda3/envs/scDOT/lib/python3.12/site-packages/scanpy/preprocessing/_scale.py:318: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


-------------------------------
单个标签的运输成本矩阵形状 (1, 1492)
单个标签的运输成本矩阵形状 (1, 1492)
单个标签的运输成本矩阵形状 (1, 1492)
单个标签的运输成本矩阵形状 (1, 1492)
单个标签的运输成本矩阵形状 (1, 1492)
单个标签的运输成本矩阵形状 (1, 1492)
单个标签的运输成本矩阵形状 (1, 1492)
单个标签的运输成本矩阵形状 (1, 1492)
单个标签的运输成本矩阵形状 (1, 1492)
单个标签的运输成本矩阵形状 (1, 1492)
单个标签的运输成本矩阵形状 (1, 1492)
单个标签的运输成本矩阵形状 (1, 1492)
单个标签的运输成本矩阵形状 (1, 1492)
-------------------------------
单个标签的运输成本矩阵形状 (1, 1492)
单个标签的运输成本矩阵形状 (1, 1492)
单个标签的运输成本矩阵形状 (1, 1492)
单个标签的运输成本矩阵形状 (1, 1492)
单个标签的运输成本矩阵形状 (1, 1492)
单个标签的运输成本矩阵形状 (1, 1492)
单个标签的运输成本矩阵形状 (1, 1492)
单个标签的运输成本矩阵形状 (1, 1492)
-------------------------------
单个标签的运输成本矩阵形状 (1, 1492)
单个标签的运输成本矩阵形状 (1, 1492)
单个标签的运输成本矩阵形状 (1, 1492)
单个标签的运输成本矩阵形状 (1, 1492)
单个标签的运输成本矩阵形状 (1, 1492)
单个标签的运输成本矩阵形状 (1, 1492)
单个标签的运输成本矩阵形状 (1, 1492)
单个标签的运输成本矩阵形状 (1, 1492)
第1个运输成本矩阵的形状为：(1492, 15)
第2个运输成本矩阵的形状为：(1492, 15)
第3个运输成本矩阵的形状为：(1492, 15)
更新初始的权重 [0.33058722 0.33171419 0.33769858]
tt [np.float64(6.435102287322398e-12), np.float64(6.41327664813696e-12), np.float64(6.2986

In [ ]:
score

array([0.35212508, 0.7921045 , 0.77682162, ..., 0.63517375, 0.58834631,
       0.95207708])

# Evaluation

In [ ]:
# file_name = "/data_ssd/ysQiu/TanWeiqiang/scDOT/datasets/pbmc/pbmc/" + f"{query_name}_label.csv"
file_name = "/data_ssd/ysQiu/TanWeiqiang/mtANN/pancreas/" + f"{query_name}_label.csv"
# y_true = pd.read_csv(filepath_or_buffer=file_name, header=0, index_col=0).iloc[:,0].values # 原数据集用
y_true = pd.read_csv(filepath_or_buffer=file_name, header=0, index_col=False).iloc[:,0].values # 换数据集用
accuracy1 = np.sum(y_hat == y_true) / y_true.shape[0]
accuracy2 = np.sum(y_hat_with_unseen == y_true) / y_true.shape[0]
print(f"The annotation accuracy of {query_name} is {accuracy1, accuracy2}")

The annotation accuracy of xin is (np.float64(0.8806970509383378), np.float64(0.6642091152815014))


In [ ]:
from sklearn.cluster import KMeans
# import numpy as np
import collections

n: int = len(set(y_hat_with_unseen))

y_pred: np.ndarray = y_true.copy()
# 应用 K-means 算法
kmeans = KMeans(n_clusters=n, random_state=314)
kmeans.fit(X=x)
y_kmeans = kmeans.labels_

# 对于每个簇，统计每个实际类型出现的次数
for cluster_id in range(n):
    # 获取当前簇的所有样本索引
    cluster_samples = np.where(y_kmeans == cluster_id)[0]
    # 获取这些样本的实际类型
    cluster_labels = y_hat_with_unseen[cluster_samples]
    
    # 统计每个类型出现的次数
    label_counts = collections.Counter(cluster_labels)
    
    # 找到出现次数最多的类型
    most_common_label = label_counts.most_common(1)[0][0]
    
    # 将这个类型赋值给簇中的所有样本
    y_pred[cluster_samples] = most_common_label

# 此时 y_hat_with_unseen 包含了每个样本根据簇中出现次数最多的实际类型重新分类的结果
res = pd.DataFrame(data={"pred2": y_pred, "pred1": y_hat_with_unseen, "true": y_true})
unseen_res = res[res["pred1"] == "unseen"]
# unseen_res = res

accuracy2 = (np.sum(y_hat_with_unseen == y_true) + np.sum(unseen_res['pred2'] == unseen_res['true'])) / y_true.shape[0]
# accuracy2 = np.sum(unseen_res['pred2'] == unseen_res['true']) / y_true.shape[0]
print(f"The annotation accuracy of {query_name} is {accuracy1, accuracy2}")

The annotation accuracy of xin is (np.float64(0.8806970509383378), np.float64(0.760053619302949))


In [ ]:
pd.DataFrame(data=y_hat_with_unseen).value_counts()

0         
alpha         511
unseen        469
beta          370
gamma          71
delta          39
ductal         16
acinar         13
macrophage      2
mast            1
Name: count, dtype: int64

In [ ]:
res

,pred2,pred1,true
0,unseen,unseen,beta
1,beta,beta,beta
2,beta,beta,beta
3,beta,beta,beta
4,beta,beta,beta
...,...,...,...
1487,gamma,gamma,gamma
1488,gamma,gamma,gamma
1489,unseen,ductal,gamma
1490,gamma,gamma,gamma


In [ ]:
score

array([0.35212508, 0.7921045 , 0.77682162, ..., 0.63517375, 0.58834631,
       0.95207708])

In [ ]:
unseen_res

,pred2,pred1,true
0,unseen,unseen,beta
7,unseen,unseen,beta
13,beta,unseen,beta
18,unseen,unseen,beta
21,unseen,unseen,beta
...,...,...,...
1453,unseen,unseen,gamma
1454,gamma,unseen,gamma
1455,unseen,unseen,gamma
1461,unseen,unseen,gamma


In [ ]:
def pred(pred1: str, pred2: str) -> str:
    return pred1 if pred1 != "unseen" else pred2


In [ ]:
res["pred"] = res.apply(func=lambda x: pred(pred1=x[1], pred2=x[0]), axis=1)

/tmp/ipykernel_19990/1751349001.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  res["pred"] = res.apply(func=lambda x: pred(pred1=x[1], pred2=x[0]), axis=1)


In [ ]:
res

,pred2,pred1,true,pred
0,unseen,unseen,beta,unseen
1,beta,beta,beta,beta
2,beta,beta,beta,beta
3,beta,beta,beta,beta
4,beta,beta,beta,beta
...,...,...,...,...
1487,gamma,gamma,gamma,gamma
1488,gamma,gamma,gamma,gamma
1489,unseen,ductal,gamma,ductal
1490,gamma,gamma,gamma,gamma


In [ ]:
res[res["pred"] == res["true"]].shape[0] / res.shape[0]

0.760053619302949

In [ ]:
set(res["true"].to_list())

{'alpha', 'beta', 'delta', 'gamma'}

In [ ]:
type_list = set(res["pred"].to_list())
# type_list.add("Megakaryocytes")
type_list = type_list.union(set(res["true"].to_list()))
type_list

{'acinar',
 'alpha',
 'beta',
 'delta',
 'ductal',
 'gamma',
 'macrophage',
 'mast',
 'unseen'}

In [ ]:
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

In [ ]:
report = classification_report(y_true=res['true'], y_pred=res['pred'], target_names=type_list)
print(report)

              precision    recall  f1-score   support

      unseen       0.00      0.00      0.00         0
        mast       1.00      0.67      0.81       886
       gamma       1.00      0.89      0.94       472
      acinar       0.98      0.84      0.90        49
       alpha       0.00      0.00      0.00         0
      ductal       1.00      0.87      0.93        85
       delta       0.00      0.00      0.00         0
  macrophage       0.00      0.00      0.00         0
        beta       0.00      0.00      0.00         0

    accuracy                           0.76      1492
   macro avg       0.44      0.36      0.40      1492
weighted avg       1.00      0.76      0.86      1492



/data_ssd/ysQiu/anaconda3/envs/scDOT/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/data_ssd/ysQiu/anaconda3/envs/scDOT/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/data_ssd/ysQiu/anaconda3/envs/scDOT/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu

In [ ]:
# pd.DataFrame(data=x).to_csv(path_or_buf='./modified.csv', index=False)
# pd.DataFrame(data=M).to_csv(path_or_buf='./cost_modified.csv', index=False)

In [ ]:
'''
# from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

data = pd.DataFrame(np.concatenate([x, np.reshape(a=y_hat_with_unseen, newshape=(-1,1)), np.reshape(a=y_true, newshape=(-1,1))], axis=1))
n: int = len(set(y_hat_with_unseen))
unseen = np.array(data[data[9] == 'unseen'])[:,0:-2]
y_test = np.array(data[data[9] == 'unseen'][10])

# def custom_distance(x: np.ndarray, y: np.ndarray):
#     # 自定义距离
#     return 1 - (x @ y) / (np.linalg.norm(x=x) * np.linalg.norm(x=y))

# k: int = 1
# max_accuracy: float = 0.0
# while k <= 200:
#     knn = KNeighborsClassifier(n_neighbors=k, metric=custom_distance)
#     knn.fit(X=x, y=y_hat_with_unseen)

#     y_pred = knn.predict(unseen)

#     # 计算准确率
#     accuracy = accuracy_score(y_test, y_pred)
#     print(f'Accuracy: {accuracy:.2f}', k)
#     # if accuracy >= max_accuracy:
#     #     max_accuracy = accuracy
#     #     k += 1
#     #     continue
#     # break
#     k += 1
'''

"\n# from sklearn.model_selection import train_test_split\nfrom sklearn.neighbors import KNeighborsClassifier\nfrom sklearn.metrics import accuracy_score\n\ndata = pd.DataFrame(np.concatenate([x, np.reshape(a=y_hat_with_unseen, newshape=(-1,1)), np.reshape(a=y_true, newshape=(-1,1))], axis=1))\nn: int = len(set(y_hat_with_unseen))\nunseen = np.array(data[data[9] == 'unseen'])[:,0:-2]\ny_test = np.array(data[data[9] == 'unseen'][10])\n\n# def custom_distance(x: np.ndarray, y: np.ndarray):\n#     # 自定义距离\n#     return 1 - (x @ y) / (np.linalg.norm(x=x) * np.linalg.norm(x=y))\n\n# k: int = 1\n# max_accuracy: float = 0.0\n# while k <= 200:\n#     knn = KNeighborsClassifier(n_neighbors=k, metric=custom_distance)\n#     knn.fit(X=x, y=y_hat_with_unseen)\n\n#     y_pred = knn.predict(unseen)\n\n#     # 计算准确率\n#     accuracy = accuracy_score(y_test, y_pred)\n#     print(f'Accuracy: {accuracy:.2f}', k)\n#     # if accuracy >= max_accuracy:\n#     #     max_accuracy = accuracy\n#     #     k += 